In [4]:
#%run Untitled5.ipynb import intents

In [ ]:
import nltk
import torch
import numpy as np
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
def stem(word):
    return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence,all_words):
    tokenized_sentence=[stem(w) for w in tokenized_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx]=1.0
    return bag
sentence=["hello","how","are","you"]
words=["hi","hello","I","you","bye","thanks","cool"]
bag=bag_of_words(sentence,words)
print(bag)
a="Hi there, how can I help?"
print(a)
a=tokenize(a)
print(a)

In [20]:
'''words=["organ","organise","organisation","organdonor"]
stemmed_Words=[stem(w) for w in words]
print(stemmed_Words)'''

['organ', 'organis', 'organis', 'organdonor']


In [ ]:
'''
words2=["organize","organizes","organizing"]
stemmed_Words2=[stem(w) for w in words2]
print(stemmed_Words2) '''

In [ ]:
import json
intents = json.loads(open('intents.json', encoding='utf-8').read())

In [ ]:
print(intents)

In [47]:
import torch
import torch.nn as nn
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.l1=nn.Linear(input_size,hidden_size)
        self.l2=nn.Linear(hidden_size,hidden_size)
        self.l3=nn.Linear(hidden_size,num_classes)
        self.relu=nn.ReLU()
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu(out)
        out=self.l3(out)
        return out

In [52]:
import numpy as np
import random
import json

import torch
import torch.nn as nn

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data={
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags
}


26 patterns
7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
54 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
54 7
Epoch [100/1000], Loss: 1.6405
Epoch [200/1000], Loss: 0.1731
Epoch [300/1000], Loss: 0.0712
Epoch [400/1000], Loss: 0.0266
Epoch [500/1000], Loss: 0.0046
Epoch [600/1000], Loss: 0.0063
Epoch [700/1000], Loss: 0.0059
Epoch [800/1000], Loss: 0.0010
Epoch [900/1000], Loss: 0.0030
Epoch [1000/1000], Loss: 0.0003
final loss: 0.0003


In [51]:
FILE="data.pth"
torch.save(data,FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [44]:
import sys
print(sys.executable)

C:\Anaconda\python.exe


In [45]:
sys.path

['C:\\Users\\ANJANI SAMHITHA',
 'C:\\Anaconda\\python38.zip',
 'C:\\Anaconda\\DLLs',
 'C:\\Anaconda\\lib',
 'C:\\Anaconda',
 '',
 'C:\\Anaconda\\lib\\site-packages',
 'C:\\Anaconda\\lib\\site-packages\\locket-0.2.1-py3.8.egg',
 'C:\\Anaconda\\lib\\site-packages\\win32',
 'C:\\Anaconda\\lib\\site-packages\\win32\\lib',
 'C:\\Anaconda\\lib\\site-packages\\Pythonwin',
 'C:\\Anaconda\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\ANJANI SAMHITHA\\.ipython']

In [16]:
class NeuralNet(nn.Module):
    def _init_(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self)._init_()
        self.l1=nn.Linear(input_size,hidden_size)
        self.l2=nn.Linear(hidden_size,hidden_size)
        self.l3=nn.Linear(hidden_size,num_classes)
        self.relu=nn.ReLU()
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu(out)
        out=self.l3(out)
        return out

In [ ]:
import random
import json
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('intents.json','r') as f:
    intents=json.load(f)
FILE="data.pth"
data=torch.load(FILE)
input_size=data["input_size"]
hidden_size=data["hidden_size"]
output_size=data["output_size"]
all_words=data["all_words"]
tags=data["tags"]
model_state=data["model_state"]
model=NeuralNet(input_size,hidden_size,output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name="Liam"
print("Let's chat! type 'quit' to exit")
while True:
    sentence=input('You: ')
    if sentence=="quit":
        break
    sentence=tokenize(sentence)
    X=bag_of_words(sentence,all_words)
    X=X.reshape(1,X.shape[0])
    X=torch.from_numpy(X).to(device)
    
    output=model(X)
    _, predicted=torch.max(output,dim=1)
    tag=tags[predicted.item()]
    
    probs=torch.softmax(output,dim=1)
    prob=probs[0][predicted.item()]
    
    if prob.item()>0.75:
        for intent in intents["intents"]:
            if tag==intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}:I donot understand...")
    

Let's chat! type 'quit' to exit
You: hi
Liam: Hey :-)
You: what do you sell
Liam: We sell coffee and tea
You: how long shipping takes?
Liam: Shipping takes 2-4 days
You: can you help me?
Liam:I donot understand...
You: bye
Liam: See you later, thanks for visiting
